## The Hetionet data contains the source node and a target node with the kind of relation that exist between them. let's start by extracting the nodes IDs for further processing.


In [4]:
import pandas as pd
edge_data = pd.read_csv("data/hetionet-v1.0-edges.sif", sep = "\t")
node_data = pd.read_csv("data/hetionet-v1.0-nodes.tsv", sep = "\t")
meta_edge_data = pd.read_csv("data/metaedges.tsv", sep = "\t")
print(edge_data.head(5))
print(node_data.head(5))
print(meta_edge_data.head(5))

        source metaedge                          target
0   Gene::9021     GpBP  Biological Process::GO:0071357
1  Gene::51676     GpBP  Biological Process::GO:0098780
2     Gene::19     GpBP  Biological Process::GO:0055088
3   Gene::3176     GpBP  Biological Process::GO:0010243
4   Gene::3039     GpBP  Biological Process::GO:0006898
                        id                       name     kind
0  Anatomy::UBERON:0000002             uterine cervix  Anatomy
1  Anatomy::UBERON:0000004                       nose  Anatomy
2  Anatomy::UBERON:0000006        islet of Langerhans  Anatomy
3  Anatomy::UBERON:0000007            pituitary gland  Anatomy
4  Anatomy::UBERON:0000010  peripheral nervous system  Anatomy
                          metaedge abbreviation   edges  source_nodes  \
0   Anatomy - downregulates - Gene          AdG  102240            36   
1       Anatomy - expresses - Gene          AeG  526407           241   
2     Anatomy - upregulates - Gene          AuG   97848            

In [5]:
edge_data.head(5)

,source,metaedge,target
0,Gene::9021,GpBP,Biological Process::GO:0071357
1,Gene::51676,GpBP,Biological Process::GO:0098780
2,Gene::19,GpBP,Biological Process::GO:0055088
3,Gene::3176,GpBP,Biological Process::GO:0010243
4,Gene::3039,GpBP,Biological Process::GO:0006898


In [6]:
import os
import pandas as pd

def create_entity_relation_dicts_from_df(data_path, triples_df):
    entity_set = set(triples_df['source']).union(set(triples_df['target']))
    relation_set = set(triples_df['metaedge'])

    entity2id = {entity: idx for idx, entity in enumerate(entity_set)}
    relation2id = {relation: idx for idx, relation in enumerate(relation_set)}

    with open(os.path.join(data_path, 'entities.dict'), 'w') as fout:
        for entity, idx in entity2id.items():
            fout.write(f"{idx}\t{entity}\n")

    with open(os.path.join(data_path, 'relations.dict'), 'w') as fout:
        for relation, idx in relation2id.items():
            fout.write(f"{idx}\t{relation}\n")

    return entity2id, relation2id

# Example usage:
data_path = 'data/FB15k'
triples_df = pd.read_csv("data/FB15k/hetionet-v1.0-edges.sif", delimiter="\t")
entity2id, relation2id = create_entity_relation_dicts_from_df(data_path, triples_df)

In [7]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
# Split the dataset into train, validation, and test sets
train_data, test_data = train_test_split(edge_data, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

# Function to save DataFrame to a text file
def save_to_text_file(df, file_name):
    df.to_csv(file_name, sep='\t', index=False, header = False)

# Save train, validation, and test datasets to text files
path = "data/FB15k/"
save_to_text_file(train_data, path + 'train.txt')
save_to_text_file(val_data, path + 'valid.txt')
save_to_text_file(test_data, path + 'test.txt')

# Descriptive Statistics


# Nodes

In [8]:
print(f"number of nodes: {len(node_data)}")
print("Number of unique node types: {}".format(len(node_data["kind"].unique())))
print("-"*40)
for node_type in node_data["kind"].unique():
    print(node_type)
print("-"*40)

number of nodes: 47031
Number of unique node types: 11
----------------------------------------
Anatomy
Biological Process
Cellular Component
Compound
Disease
Gene
Molecular Function
Pathway
Pharmacologic Class
Side Effect
Symptom
----------------------------------------


# Convert dataset to tripples format

In [9]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import json
import logging
import os
import random

import numpy as np
import torch
from torch.utils.data import DataLoader

from model import KGEModel
from dataloader import TrainDataset, BidirectionalOneShotIterator

# Define parameters

In [13]:
class Args:
    def __init__(self):
        self.cuda = False
        self.do_train = True
        self.do_valid = True
        self.do_test = True
        self.evaluate_train = False
        self.countries = False
        self.regions = None
        self.data_path = 'data/FB15k'
        self.model = 'TransE'
        self.double_entity_embedding = False
        self.double_relation_embedding = False
        self.negative_sample_size = 128
        self.hidden_dim = 200
        self.gamma = 12.0
        self.negative_adversarial_sampling = False
        self.adversarial_temperature = 1.0
        self.batch_size = 1024
        self.regularization = 0.0
        self.test_batch_size = 4
        self.uni_weight = False
        self.learning_rate = 0.0001
        self.cpu_num = 10
        self.init_checkpoint = None
        self.save_path = 'models/TransE_FB15k'
        self.max_steps = 100
        self.warm_up_steps = None
        self.save_checkpoint_steps = 1000
        self.valid_steps = 1000
        self.log_steps = 100
        self.test_log_steps = 1000
        self.nentity = 0
        self.nrelation = 0

args = Args()

# Main function

In [14]:
def read_triple(file_path, entity2id, relation2id):
    triples = []
    with open(file_path) as fin:
        for line in fin:
            head, relation, tail = line.strip().split('\t')
            triples.append((entity2id[head], relation2id[relation], entity2id[tail]))
    return triples

def set_logger(args):
    log_file = os.path.join(args.save_path or args.init_checkpoint, 'train.log')
    logging.basicConfig(
        format='%(asctime)s %(message)s',
        level=logging.INFO,
        handlers=[
            logging.FileHandler(log_file),
            logging.StreamHandler()
        ]
    )

def log_metrics(mode, step, metrics):
    for metric in metrics:
        logging.info('%s %s at step %d: %f' % (mode, metric, step, metrics[metric]))

In [ ]:
def override_config(args):
    # Implement the logic to override config based on checkpoint
    pass

def save_model(model, optimizer, save_variable_list, args):
    # Implement the logic to save the model
    pass
def main(args):
    if (not args.do_train) and (not args.do_valid) and (not args.do_test):
        raise ValueError('One of train/val/test mode must be chosen.')
    
    if args.init_checkpoint:
        override_config(args)
    elif args.data_path is None:
        raise ValueError('One of init_checkpoint/data_path must be chosen.')

    if args.do_train and args.save_path is None:
        raise ValueError('Where do you want to save your trained model?')
    
    if args.save_path and not os.path.exists(args.save_path):
        os.makedirs(args.save_path)
    
    set_logger(args)
    
    with open(os.path.join(args.data_path, 'entities.dict')) as fin:
        entity2id = dict()
        for line in fin:
            eid, entity = line.strip().split('\t')
            entity2id[entity] = int(eid)

    with open(os.path.join(args.data_path, 'relations.dict')) as fin:
        relation2id = dict()
        for line in fin:
            rid, relation = line.strip().split('\t')
            relation2id[relation] = int(rid)
    
    if args.countries:
        regions = list()
        with open(os.path.join(args.data_path, 'regions.list')) as fin:
            for line in fin:
                region = line.strip()
                regions.append(entity2id[region])
        args.regions = regions

    nentity = len(entity2id)
    nrelation = len(relation2id)
    
    args.nentity = nentity
    args.nrelation = nrelation
    
    logging.info('Model: %s' % args.model)
    logging.info('Data Path: %s' % args.data_path)
    logging.info('#entity: %d' % nentity)
    logging.info('#relation: %d' % nrelation)
    
    train_triples = read_triple(os.path.join(args.data_path, 'train.txt'), entity2id, relation2id)
    logging.info('#train: %d' % len(train_triples))
    valid_triples = read_triple(os.path.join(args.data_path, 'valid.txt'), entity2id, relation2id)
    logging.info('#valid: %d' % len(valid_triples))
    test_triples = read_triple(os.path.join(args.data_path, 'test.txt'), entity2id, relation2id)
    logging.info('#test: %d' % len(test_triples))
    
    all_true_triples = train_triples + valid_triples + test_triples
    
    kge_model = KGEModel(
        model_name=args.model,
        nentity=nentity,
        nrelation=nrelation,
        hidden_dim=args.hidden_dim,
        gamma=args.gamma,
        double_entity_embedding=args.double_entity_embedding,
        double_relation_embedding=args.double_relation_embedding
    )
    
    logging.info('Model Parameter Configuration:')
    for name, param in kge_model.named_parameters():
        logging.info('Parameter %s: %s, require_grad = %s' % (name, str(param.size()), str(param.requires_grad)))

    if args.cuda:
        kge_model = kge_model.cuda()
    
    if args.do_train:
        train_dataloader_head = DataLoader(
            TrainDataset(train_triples, nentity, nrelation, args.negative_sample_size, 'head-batch'), 
            batch_size=args.batch_size*2,
            shuffle=True, 
            num_workers=max(1, args.cpu_num//2),
            collate_fn=TrainDataset.collate_fn
        )
        
        train_dataloader_tail = DataLoader(
            TrainDataset(train_triples, nentity, nrelation, args.negative_sample_size, 'tail-batch'), 
            batch_size=args.batch_size*2,
            shuffle=True, 
            num_workers=max(1, args.cpu_num//2),
            collate_fn=TrainDataset.collate_fn
        )
        
        train_iterator = BidirectionalOneShotIterator(train_dataloader_head, train_dataloader_tail)
        
        current_learning_rate = args.learning_rate
        optimizer = torch.optim.Adam(
            filter(lambda p: p.requires_grad, kge_model.parameters()), 
            lr=current_learning_rate
        )
        if args.warm_up_steps:
            warm_up_steps = args.warm_up_steps
        else:
            warm_up_steps = args.max_steps // 2

    if args.init_checkpoint:
        logging.info('Loading checkpoint %s...' % args.init_checkpoint)
        checkpoint = torch.load(os.path.join(args.init_checkpoint, 'checkpoint'))
        init_step = checkpoint['step']
        kge_model.load_state_dict(checkpoint['model_state_dict'])
        if args.do_train:
            current_learning_rate = checkpoint['current_learning_rate']
            warm_up_steps = checkpoint['warm_up_steps']
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    else:
        logging.info('Randomly Initializing %s Model...' % args.model)
        init_step = 0

    #Training loop
    step = init_step
    #logging initial parameters
    logging.info('Start Training...')
    logging.info('init_step = %d' % init_step)
    logging.info('batch_size = %d' % args.batch_size)
    logging.info('negative_adversarial_sampling = %d' % args.negative_adversarial_sampling)
    logging.info('hidden_dim = %d' % args.hidden_dim)
    logging.info('gamma = %f' % args.gamma)
    logging.info('negative_adversarial_sampling = %s' % str(args.negative_adversarial_sampling))
    if args.negative_adversarial_sampling:
        logging.info('adversarial_temperature = %f' % args.adversarial_temperature)

    #Checking training condition
    if args.do_train:
        logging.info('learning_rate = %d' % current_learning_rate)

        training_logs = []

        #training loop initialise from initial stem to maximum step
        for step in range(init_step, args.max_steps):
            #Trianing step: calls the training model in a single step and return the log metrics
            log = kge_model.train_step(kge_model, optimizer, train_iterator, args)
            
            training_logs.append(log)
            
            #learning rate adjustment
           # if step >= warm_up_steps: #warm up steps should always be lower
           #     current_learning_rate = current_learning_rate / 10
           #     logging.info('Change learning_rate to %f at step %d' % (current_learning_rate, step))
           #     #creates a new adam optimizer with the new leatning rate
           #     optimizer = torch.optim.Adam(
           #         filter(lambda p: p.requires_grad, kge_model.parameters()), 
           #         lr=current_learning_rate
           #     )
           #     warm_up_steps = warm_up_steps * 3
            #save checkpoints
            if step % args.save_checkpoint_steps == 0:
                save_variable_list = {
                    'step': step, 
                    'current_learning_rate': current_learning_rate,
                    'warm_up_steps': warm_up_steps
                }
                save_model(kge_model, optimizer, save_variable_list, args)
                
            if step % args.log_steps == 0:
                metrics = {}
                for metric in training_logs[0].keys():
                    metrics[metric] = sum([log[metric] for log in training_logs])/len(training_logs)
                log_metrics('Training average', step, metrics)
                training_logs = []
                
            if args.do_valid and step % args.valid_steps == 0:
                logging.info('Evaluating on Valid Dataset...')
                metrics = kge_model.test_step(kge_model, valid_triples, all_true_triples, args)
                log_metrics('Valid', step, metrics)
        
        save_variable_list = {
            'step': step, 
            'current_learning_rate': current_learning_rate,
            'warm_up_steps': warm_up_steps
        }
        save_model(kge_model, optimizer, save_variable_list, args)
        
    if args.do_valid:
        logging.info('Evaluating on Valid Dataset...')
        metrics = kge_model.test_step(kge_model, valid_triples, all_true_triples, args)
        log_metrics('Valid', step, metrics)
    
    if args.do_test:
        logging.info('Evaluating on Test Dataset...')
        metrics = kge_model.test_step(kge_model, test_triples, all_true_triples, args)
        log_metrics('Test', step, metrics)
    
    if args.evaluate_train:
        logging.info('Evaluating on Training Dataset...')
        metrics = kge_model.test_step(kge_model, train_triples, all_true_triples, args)
        log_metrics('Test', step, metrics)

# Initialize the argument class
args = Args()

# Run the main function with these arguments
main(args)

2024-06-02 02:37:11,715 Model: TransE
2024-06-02 02:37:11,762 Data Path: data/FB15k
2024-06-02 02:37:11,776 #entity: 45158
2024-06-02 02:37:11,780 #relation: 24
2024-06-02 02:37:21,758 #train: 1440125
2024-06-02 02:37:24,662 #valid: 360032
2024-06-02 02:37:25,796 #test: 450040
2024-06-02 02:37:26,613 Model Parameter Configuration:
2024-06-02 02:37:26,678 Parameter gamma: torch.Size([1]), require_grad = False
2024-06-02 02:37:26,680 Parameter embedding_range: torch.Size([1]), require_grad = False
2024-06-02 02:37:26,694 Parameter entity_embedding: torch.Size([45158, 200]), require_grad = True
2024-06-02 02:37:26,695 Parameter relation_embedding: torch.Size([24, 200]), require_grad = True
2024-06-02 02:38:32,592 Randomly Initializing TransE Model...
2024-06-02 02:38:32,592 Start Training...
2024-06-02 02:38:32,614 init_step = 0
2024-06-02 02:38:32,621 batch_size = 1024
2024-06-02 02:38:32,623 negative_adversarial_sampling = 0
2024-06-02 02:38:32,626 hidden_dim = 200
2024-06-02 02:38:32,6